##### Imports

In [ ]:
import re
import copy
import shutil
from pathlib import Path

# RDKit as conformer generator
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign, Draw
from IPython.display import display, Markdown

# OPI imports for performing ORCA calculations and reading output
from opi.core import Calculator
from opi.input import simple_keywords
from opi.input.structures.structure import Structure
from opi.input.simple_keywords import BasisSet, Dft, ForceField, Scf, Task, Docker, SolvationModel, Solvent 
from opi.input.simple_keywords.goat import Goat
from opi.input.blocks import BlockDocker , BlockGeom, TSMode, Hybrid, geom_wrapper 
from opi.input import blocks
from opi.input.blocks.block_geom import Scan
from opi.input.blocks.base import Block, InputFilePath
# for plotting results and visualization of molecules
from opi.input.blocks import BlockGeom, Hybrid
from opi.input.blocks.base import NumList
import py3Dmol
import pandas as pd
import matplotlib.pyplot as plt

##### Utility functions

In [ ]:
from pathlib import Path
from typing import Union, Sequence
import py3Dmol
from IPython.display import display, Markdown


def show_structure(
    structure: Union[Structure, str, Path],
    title: str = "Structure",
    stick_radius: float = 0.1,
    sphere_scale: float = 0.2,
    label_level: int = 2,
    wireframe_indices: list[int] = None,
) -> None:
    """
    Visualize a Structure or XYZ file with optional atom/bond labels and wireframe styling using py3Dmol.

    Parameters:
        structure (Structure | str | Path): Structure object or path to .xyz file.
        title (str): Title to display.
        stick_radius (float): Stick style bond radius.
        sphere_scale (float): Atom sphere scaling.
        label_level (int):
            0 = no labels,
            1 = show atom symbol + index,
            2 = as above, plus bond lengths.
        wireframe_indices (list[int]): Atom indices whose spheres will be downsized to stick radius.
    """
    if isinstance(structure, (str, Path)):
        path = Path(structure).resolve()
        if not path.exists():
            raise FileNotFoundError(f"File not found: {path}")
        structure = Structure.from_xyz(path)

    xyz_block = structure.to_xyz_block().splitlines()
    num_atoms = int(xyz_block[0])
    atom_lines = xyz_block[2:2 + num_atoms]

    atoms = []
    coords = []
    for line in atom_lines:
        parts = line.split()
        atoms.append(parts[0])
        coords.append([float(x) for x in parts[1:4]])

    mol = Chem.RWMol()
    for symbol in atoms:
        mol.AddAtom(Chem.Atom(symbol))
    conf = Chem.Conformer(len(atoms))
    for i, (x, y, z) in enumerate(coords):
        conf.SetAtomPosition(i, rdGeometry.Point3D(x, y, z))
    mol.AddConformer(conf)

    # Infer bonds
    for i in range(len(atoms)):
        for j in range(i + 1, len(atoms)):
            dist = conf.GetAtomPosition(i).Distance(conf.GetAtomPosition(j))
            ri = Chem.GetPeriodicTable().GetRcovalent(mol.GetAtomWithIdx(i).GetAtomicNum())
            rj = Chem.GetPeriodicTable().GetRcovalent(mol.GetAtomWithIdx(j).GetAtomicNum())
            if dist < 1.3 * (ri + rj):
                mol.AddBond(i, j, Chem.BondType.SINGLE)
    mol = mol.GetMol()

    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(structure.to_xyz_block(), "xyz")
    viewer.setBackgroundColor("white")

    # Global style
    viewer.setStyle({"stick": {"radius": stick_radius}, "sphere": {"scale": sphere_scale}})

    # Override wireframe atoms
    if wireframe_indices:
        viewer.setStyle(
            {"serial": [i for i in wireframe_indices]},
            {
                "stick": {"radius": stick_radius},
                "sphere": {"scale": 0.0},  # override to match stick radius
            },
        )

    # Atom labels
    if label_level >= 1:
        for atom in mol.GetAtoms():
            idx = atom.GetIdx()
            pos = conf.GetAtomPosition(idx)
            label = f"{atom.GetSymbol()}{idx}"
            viewer.addLabel(
                label,
                {
                    "position": {"x": pos.x, "y": pos.y, "z": pos.z},
                    "fontSize": 18,
                    "fontColor": "black",
                    "backgroundColor": "white",
                    "showBackground": False,
                    "id": f"{title.lower()}_atom_{idx}",
                },
            )

    # Bond length labels
    if label_level == 2:
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            p1 = conf.GetAtomPosition(i)
            p2 = conf.GetAtomPosition(j)
            mid = {"x": (p1.x + p2.x) / 2, "y": (p1.y + p2.y) / 2, "z": (p1.z + p2.z) / 2}
            dist = p1.Distance(p2)
            viewer.addLabel(
                f"{dist:.2f}",
                {
                    "position": mid,
                    "fontSize": 18,
                    "fontColor": "black",
                    "backgroundColor": "white",
                    "showBackground": False,
                },
            )

    viewer.zoomTo()
    display(Markdown(f"### {title}"))
    viewer.show()


In [19]:

from typing import Union, Optional, Sequence
from pathlib import Path

from typing import Union, Optional, Sequence
from pathlib import Path

def setup_calc(
    working_dir: Optional[Union[Path, str]],
    structure: Union[Structure, str],
    basename: str = "single_molecule_calc",
    sk_list: Optional[Sequence[str]] = None,
    bk_list: Optional[Union[Block, Sequence[Block]]] = None,
    ncores: int = 12,
    memory: int = 2200,
    visualize: bool = True,
) -> Calculator:
    """
    Set up a single-molecule ORCA calculation using a Structure, path to XYZ file, or SMILES string.

    Parameters:
        working_dir (Path or str or None): Directory for input/output files.
        structure (Structure or path or SMILES): The molecule.
        basename (str): Base name for calculation.
        sk_list (list[str]): Simple keywords for ORCA input.
        bk_list (Block or list[Block] or None): Additional ORCA input blocks.
        ncores (int): Number of CPU cores.
        memory (int): Memory in MB.
        visualize (bool): Whether to visualize the structure.

    Returns:
        Calculator: Configured Calculator instance.
    """
    # Normalize and create working directory
    if isinstance(working_dir, str):
        working_dir = Path(working_dir)
    if working_dir is None:
        working_dir = Path.cwd() / basename
    working_dir = working_dir.resolve()
    working_dir.mkdir(parents=True, exist_ok=True)

    # Convert input to Structure
    if isinstance(structure, Structure):
        mol = structure
    elif isinstance(structure, str):
        path = Path(structure)
        if path.suffix == ".xyz" and path.exists():
            mol = Structure.from_xyz(path)
        else:
            # Treat as SMILES
            mol = Structure.from_smiles(structure)
    else:
        raise TypeError(f"Unsupported structure input type: {type(structure)}")

    # Save structure to XYZ
    xyz_file = working_dir / f"{basename}.xyz"
    xyz_file.write_text(mol.to_xyz_block())

    # Optional visualization
    if visualize:
        show_structure(mol, title=basename)

    # Set up Calculator
    calc = Calculator(basename=basename, working_dir=working_dir)
    calc.structure = mol
    calc.structure.charge = mol.charge
    calc.structure.multiplicity = mol.multiplicity

    if sk_list:
        calc.input.add_simple_keywords(*sk_list)

    # Add blocks
    if isinstance(bk_list, Block):
        calc.input.add_blocks(bk_list)
    elif isinstance(bk_list, Sequence):
        for bk in bk_list:
            if not isinstance(bk, Block):
                raise TypeError(f"Invalid block in bk_list: {bk}")
            calc.input.add_blocks(bk)

    # Resources
    calc.input.ncores = ncores
    calc.input.memory = memory
    calc.write_input()
    return calc


In [20]:
from pathlib import Path
from typing import Optional, Sequence, Union

def setup_docking_calc(
    working_dir: Union[str, Path],
    host_smiles: str,
    guest_smiles: str,
    basename: str = "docking_calc",
    sk_list: Optional[Sequence[str]] = ["xtb", "alpb(water)"],
    bk_list: Optional[Sequence[Block]] = None,
    ncores: int = 12,
    memory: int = 2200,
    visualize: bool = True,
) -> Calculator:
    """
    Set up an ORCA docking calculation using host and guest SMILES strings.

    Parameters:
        working_dir (str or Path): Base directory where 'docking' subdirectory will be created.
        host_smiles (str): SMILES string of the host molecule.
        guest_smiles (str): SMILES string of the guest molecule.
        basename (str): Base name for input/output files.
        sk_list (list[str]): Simple ORCA keywords.
        bk_list (list[Block] or None): Additional input blocks.
        ncores (int): Number of CPU cores.
        memory (int): Memory in MB.
        visualize (bool): Whether to visualize host and guest structures.

    Returns:
        Calculator: Configured Calculator instance.
    """
    # Normalize and prepare paths
    if isinstance(working_dir, str):
        working_dir = Path(working_dir)
    docking_dir = working_dir.resolve() / "docking"
    docking_dir.mkdir(parents=True, exist_ok=True)

    # Generate and save guest structure
    guest_structure = Structure.from_smiles(guest_smiles)
    guest_xyz_path = docking_dir / f"{guest_smiles.lower()}.xyz"
    guest_xyz_path.write_text(guest_structure.to_xyz_block())

    if visualize:
        show_structure(guest_structure, title="Guest")

    # Prepare Docker block
    docker_block = BlockDocker(guest=InputFilePath.from_string(guest_xyz_path.name))

    # Combine with optional blocks
    all_blocks = [docker_block]
    if bk_list:
        all_blocks.extend(bk_list)

    # Use setup_calc to build host-side calculator with guest block
    calc = setup_calc(
        working_dir=docking_dir,
        structure=host_smiles,
        basename=basename,
        sk_list=sk_list,
        bk_list=all_blocks,
        ncores=ncores,
        memory=memory,
        visualize=visualize,
    )

    return calc


In [ ]:
from pathlib import Path
from typing import Optional, Union, Sequence
from opi.input.blocks import Block

def run_multi_step_calculation(
    working_dir: Union[str, Path],
    structure: Union[Structure, str],
    steps: Sequence[dict],
    visualize: bool = True,
    xyz_suffix: str = ".xyz",
) -> Calculator:
    """
    Run a multi-step ORCA calculation workflow, where each step builds on the geometry of the previous.

    Parameters:
        working_dir (str | Path): Root directory for the entire calculation.
        structure (Structure | str): Initial geometry, either as a Structure or path to an XYZ file.
        steps (Sequence[dict]): List of calculation step configurations.
                                 Each step is a dict that may contain:
                                    - 'basename': str
                                    - 'sk_list': list[str]
                                    - 'bk_list': list[Block] or Block
                                    - 'ncores': int
                                    - 'memory': int
        visualize (bool): If True, display the final structure using `show_structure`.
        xyz_suffix (str): Suffix used for reading the output XYZ files.

    Returns:
        Calculator: The final Calculator instance after all steps.
    """
    if isinstance(working_dir, str):
        working_dir = Path(working_dir)
    working_dir.mkdir(parents=True, exist_ok=True)

    if isinstance(structure, str):
        structure = Structure.from_xyz(structure)

    current_structure = structure

    for i, step in enumerate(steps):
        basename = step.get("basename", f"step_{i+1}")
        step_dir = working_dir / basename
        step_dir.mkdir(parents=True, exist_ok=True)

        calc = Calculator(basename=basename, working_dir=step_dir)
        calc.structure = current_structure
        calc.structure.charge = current_structure.charge
        calc.structure.multiplicity = current_structure.multiplicity

        sk_list = step.get("sk_list", [])
        bk_list = step.get("bk_list", [])
        calc.input.add_simple_keywords(*sk_list)

        if isinstance(bk_list, Block):
            calc.input.add_blocks(bk_list)
        elif isinstance(bk_list, list):
            for bk in bk_list:
                calc.input.add_blocks(bk)

        calc.input.ncores = step.get("ncores", 12)
        calc.input.memory = step.get("memory", 2200)

        calc.write_input()
        calc.run()

        # Load geometry from output
        output_file = step_dir / f"{basename}{xyz_suffix}"
        current_structure = Structure.from_xyz(output_file)

        if visualize:
            show_structure(current_structure, title=basename)

    return calc


## BnOH + H2O2

In [16]:
# Define SMILES for host and guest
host = "c1ccccc1CO"  # Benzyl alcohol
guest = "OO"         # Hydrogen peroxide

# Define base working directory
base_dir = Path("BnOH_plus_H2O2")

### Docking reactants

In [41]:
# Set up docking calculation
calc = setup_docking_calc(
    working_dir=base_dir,
    host_smiles=host,
    guest_smiles=guest,
    basename="BnOH_H2O2",
    visualize=True,
)

### Guest

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### BnOH_H2O2

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# Define SMILES and human-readable names
smiles_dict = {
    "c1ccccc1CO": "BnOH",   # Benzyl alcohol
    "OO": "H2O2",           # Hydrogen peroxide
}
smiles = list(smiles_dict.keys())

# Construct reaction name and base working directory
reaction_name = f"{smiles[0]}_+_{smiles[-1]}"
base_dir = Path(reaction_name)

# Clean and create base directory
# if base_dir.exists():
#     shutil.rmtree(base_dir)
# base_dir.mkdir(parents=True)

### Dock reactants

In [158]:
show_structure("/home/madsr2d2/chem/BnOH_plus_H2O2/docking/BnOH_H2O2.docker.xyz", wireframe_indices=[0, 1, 2])

### Structure

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [19]:
# Create docking subdirectory
docking_dir = base_dir / "docking"
if docking_dir.exists():
    shutil.rmtree(docking_dir)
docking_dir.mkdir(parents=True)

# Set up docking calculation
sk_list = ["xtb alpb(water)"]
calc = setup_docker_from_smiles(
    working_dir=docking_dir,
    sk_list=sk_list,
    smiles=smiles,
)

### Host

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Guest

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [20]:
# Run calculation
calc.run()

In [52]:
# docked structure
output = calc.get_output()
docked_structure = Structure.from_xyz(docking_dir / f"{output.basename}.docker.xyz")
# Show final geometry
show_structure(docked_structure, title=f"{output.basename}.docker.xyz")

NameError: name 'docking_dir' is not defined

### Scan reaction coordinate

In [ ]:
# Create docking subdirectory

blocks.BlockGeom(cartfallback=True, constraints=BlockGeom.constraint_str(["strings="]))


scan_dir = Path.cwd() / "scan"
scan_dir.mkdir(parents=True)
sk_list = ["xtb alpb(water)", "opt"]

# Parse scan string to Scan object
# scan = BlockGeom.scan_str("B 13 17 = 4, 0.96, 10")
# BlockGeom.ts_active_atoms
# ts_str = BlockGeom.tsmode_str()
# constraint = BlockGeom.constraint_str("B 13 16 = 4, 0.  96, 10")

# active_atomes = Hybrid
# active_atomes.atom1 = 13
# active_atomes.atom2 = 17
# active_atomes.atom3 = 6
# # Create BlockGeom and assign scan
geom_block = BlockGeom()
# geom_block.scan = scan
# geom_block.fullscan = True
# geom_block.ts_active_atoms = Hybrid(mode="A",list=NumList([13, 17, 6, 16]))
geom_block.constraints = BlockGeom.constraint_str(["a 6 13 17 175.0 C", "B 13 17 2.0 C" , "a 13 17 16 175.0 c"])
# geom_block.coordsys = "cartesian"

# # geom_block.ts_mode = TSMode({'atom1': 13,'atom2': 17,'atom3':6})  # Active atoms for TS scan

# Assemble block list
bk_list = [geom_block]

# Run calculation setup
calc1 = setup_single_structure_calc(
    working_dir=Path.cwd() / ,
    sk_list=sk_list,
    bk_list=bk_list,
    structure=docked_structure,
    basename=reaction_name + "_scan",
)

NameError: name 'docked_structure' is not defined

In [140]:
# Run calculation
calc1.run()

In [141]:
# docked structure
output = calc1.get_output()
docked_structure = Structure.from_xyz(scan_dir / f"{output.basename}.xyz")
# Show final geometry
show_structure(docked_structure, title=f"{output.basename}.xyz")

### c1ccccc1CO_+_OO_scan.xyz

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### XTB-TS 

In [ ]:

# Create docking subdirectory
scan_dir = base_dir / "ts"
scan_dir.mkdir(parents=True)
sk_list = ["xtb alpb(water) opt"]

# Parse scan string to Scan object
scan = BlockGeom.scan_str("B 13 16 = 4, 0.96, 10")
# constraint = BlockGeom.constraint_str("B 13 16 = 4, 0.96, 10")

# Create BlockGeom and assign scan
geom_block = BlockGeom()
geom_block.scan = scan

# Assemble block list
bk_list = [geom_block]

# Run calculation setup
calc1 = setup_single_structure_calc(
    working_dir=scan_dir,
    sk_list=sk_list,
    bk_list=bk_list,
    structure=docked_structure,
    basename=reaction_name + "_scan",
)